# Walmart Payday Pulse Forecasting (Unified Notebook)

Notebook này thực hiện pipeline hoàn chỉnh:
1. Load và merge dataset Walmart
2. Làm sạch dữ liệu và tạo calendar cơ bản
3. Decomposition bằng LightGBM để tách các thành phần phổ biến
4. Phân tích residual theo phase trong tháng
5. Fit chu kì lương 2 đỉnh bằng 2 Gaussian trên residual mean
6. Tạo feature `pay_peak_1` và `pay_peak_2`
7. Thêm lag và rolling
8. Train hai model LightGBM: không có và có payday features và so sánh RMSE

Chỉnh biến `DATA_DIR` cho đúng thư mục chứa `train.csv`, `stores.csv`, `features.csv`.

Dưới đây là **tóm tắt ngắn – súc tích – rõ ràng** của toàn bộ pipeline unified Walmart Payday Pulse:

---

# **TÓM TẮT PIPELINE WALMART PAYDAY PULSE**

## **1. Load và merge dữ liệu**

* Đọc `train.csv`, `stores.csv`, `features.csv`.
* Chuẩn hóa Date, đổi thành `WeekEndDate` (thứ Sáu).
* Merge ba bảng thành một DataFrame duy nhất `df`.

---

## **2. Làm sạch dữ liệu**

* Điền NA cho MarkDown1–5, tạo `md_sum`.
* Xử lý Weekly_Sales âm → tạo returns_flag → clip về 0.
* Tạo các cột thời gian: year, month, day, days_in_month.
* Tạo `month_phase` = day / days_in_month (vị trí tuần trong tháng).
* Tạo weekofyear, week_of_month.

---

## **3. Decomposition để tách thành phần phổ biến**

Mục tiêu: loại bỏ store effects, seasonality theo tuần, markdown, thời tiết, kinh tế vĩ mô.

* Dùng LightGBM với các feature:

  * categorical: Store, Dept, weekofyear
  * numeric: md_sum, Temperature, Fuel_Price, CPI, Unemployment
* Fit model decomposition → tạo:

  * `y_hat_decomp`: phần explainable
  * `residual`: phần chưa giải thích (phần quan trọng)

> **Residual chính là nơi chu kì lương ẩn xuất hiện.**

---

## **4. Phân tích residual theo phase trong tháng**

* Chia month_phase thành 20 bin.
* Tính residual mean mỗi bin → tạo hàm f(phase).
* Đây là dạng “chu kì lương bị làm mờ”.

---

## **5. Fit mô hình hai Gaussian lên residual mean**

* Fit hàm:

  * Gaussian 1: đỉnh lương lần 1
  * Gaussian 2: đỉnh lương lần 2
* Thu được 6 tham số:

  * a1, mu1, sigma1
  * a2, mu2, sigma2

> Đây là cách **trích xuất chu kì lương population-level** mà không giả định cứng.

---

## **6. Tạo feature pay cycle “mềm”**

* Với mỗi tuần, tính:

  * `pay_peak_1` = độ gần đỉnh lương lần 1
  * `pay_peak_2` = độ gần đỉnh lương lần 2
* Chuẩn hóa về 0 đến 1.

---

## **7. Thêm lag & rolling**

* lag_sales_t_1, t_4, t_52
* rolling_mean_sales_4

Mục tiêu: mô hình hóa inertia và annual seasonality.

---

## **8. Train hai model LightGBM**

### **Model A (baseline)**

* Không dùng pay cycle
* Chỉ dùng md_sum, weather, CPI, lag, rolling

### **Model B (payday pulse)**

* Dùng tất cả feature của Model A
* Thêm `pay_peak_1`, `pay_peak_2`

---

## **9. So sánh RMSE**

* Nếu **Model B < Model A** → chứng minh:

  * Chu kì lương hai đỉnh là thật
  * Feature pay_cycle giúp dự báo
  * Decomposition + residual mining là đúng hướng




In [ ]:
# 0. Import libraries
import os
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from scipy.optimize import curve_fit

print("Libraries loaded!")

Libraries loaded!


## 1. Load dữ liệu Walmart

Giả sử folder chứa 3 file gốc: `train.csv`, `stores.csv`, `features.csv`.

In [ ]:
!git clone https://github.com/coderbian/Datathon---Data2U.git

Cloning into 'Datathon---Data2U'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 51 (delta 12), reused 46 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 31.62 MiB | 15.54 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
# Đường dẫn dữ liệu, chỉnh lại nếu cần
TRAIN_PATH = "/content/Datathon---Data2U/data/train.csv"
FEATURES_PATH = "/content/Datathon---Data2U/data/features.csv"
STORES_PATH = "/content/Datathon---Data2U/data/stores.csv"

train = pd.read_csv(TRAIN_PATH, parse_dates=["Date"])
features = pd.read_csv(FEATURES_PATH, parse_dates=["Date"])
stores = pd.read_csv(STORES_PATH)

print("Kích thước train:", train.shape)
print("Kích thước features:", features.shape)
print("Kích thước stores:", stores.shape)

train.head()


Kích thước train: (421570, 5)
Kích thước features: (8190, 12)
Kích thước stores: (45, 3)


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


## 2. Chuẩn hóa thời gian và merge thành `df_main`

- Parse `Date` thành datetime
- Đổi tên `Date` thành `WeekEndDate` (thường là Friday)
- Merge train với stores và features

In [ ]:
# Parse Date
train["Date"] = pd.to_datetime(train["Date"])
features["Date"] = pd.to_datetime(features["Date"])

train = train.rename(columns={"Date": "WeekEndDate"})
features = features.rename(columns={"Date": "WeekEndDate"})

# Kiểm tra weekday
train["weekday"] = train["WeekEndDate"].dt.day_name()
print(train["weekday"].value_counts())

# Merge
df = (
    train.drop(columns=["weekday"])
         .merge(stores, on="Store", how="left")
         .merge(features, on=["Store", "WeekEndDate"], how="left")
)

df.head()

weekday
Friday    421570
Name: count, dtype: int64


,Store,Dept,WeekEndDate,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


## 3. Làm sạch cơ bản và tạo calendar

- Fill NA cho MarkDown1 đến MarkDown5
- Tạo `md_sum`
- Xử lý `Weekly_Sales` âm
- Thêm các cột calendar: year, month, day, days_in_month, month_phase, weekofyear, week_of_month

In [ ]:
# MarkDown xử lý NA
md_cols = [c for c in df.columns if c.startswith("MarkDown")]
if md_cols:
    df[md_cols] = df[md_cols].fillna(0)
    df["md_sum"] = df[md_cols].sum(axis=1)
else:
    df["md_sum"] = 0.0

# Weekly_Sales âm
df["returns_flag"] = (df["Weekly_Sales"] < 0).astype(int)
df["Weekly_Sales"] = df["Weekly_Sales"].clip(lower=0)

# Calendar
df["WeekEndDate"] = pd.to_datetime(df["WeekEndDate"])
df["year"] = df["WeekEndDate"].dt.year
df["month"] = df["WeekEndDate"].dt.month
df["day"] = df["WeekEndDate"].dt.day
df["days_in_month"] = df["WeekEndDate"].dt.days_in_month
df["month_phase"] = df["day"] / df["days_in_month"]
df["weekofyear"] = df["WeekEndDate"].dt.isocalendar().week.astype(int)
df["week_of_month"] = ((df["day"] - 1) // 7 + 1)

df.head()

,Store,Dept,WeekEndDate,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,IsHoliday_y,md_sum,returns_flag,year,month,day,days_in_month,month_phase,weekofyear,week_of_month
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,0.0,...,False,0.0,0,2010,2,5,28,0.178571,5,1
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,0.0,...,True,0.0,0,2010,2,12,28,0.428571,6,2
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,0.0,...,False,0.0,0,2010,2,19,28,0.678571,7,3
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,0.0,...,False,0.0,0,2010,2,26,28,0.928571,8,4
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,0.0,...,False,0.0,0,2010,3,5,31,0.161290,9,1


## 4. Decomposition: fit LightGBM để lấy residual

Mục tiêu: tách các thành phần phổ biến như Store, Dept, weekofyear, markdown, weather, CPI, Unemployment.
Phần còn lại là `residual`, nơi có thể chứa signal chu kì lương.

In [ ]:
# Xây base features cho decomposition
numeric_cols = []
for col in ["md_sum", "Temperature", "Fuel_Price", "CPI", "Unemployment"]:
    if col in df.columns:
        numeric_cols.append(col)

cat_cols = []
for col in ["Store", "Dept", "weekofyear"]:
    if col in df.columns:
        cat_cols.append(col)

df[numeric_cols] = df[numeric_cols].fillna(0)

X_decomp_input = df[numeric_cols + cat_cols].copy()

X_decomp = pd.get_dummies(
    X_decomp_input,
    columns=cat_cols,
    drop_first=True,
)

y = df["Weekly_Sales"].values

print("numeric_cols:", numeric_cols)
print("cat_cols:", cat_cols)
print("X_decomp shape:", X_decomp.shape)

numeric_cols: ['md_sum', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
cat_cols: ['Store', 'Dept', 'weekofyear']
X_decomp shape: (421570, 180)


In [ ]:
model_decomp = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

model_decomp.fit(X_decomp, y)

df["y_hat_decomp"] = model_decomp.predict(X_decomp)
df["residual"] = df["Weekly_Sales"] - df["y_hat_decomp"]

df[["Weekly_Sales", "y_hat_decomp", "residual"]].head()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010668 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1609
[LightGBM] [Info] Number of data points in the train set: 421570, number of used features: 178
[LightGBM] [Info] Start training from score 15981.467248


,Weekly_Sales,y_hat_decomp,residual
0,24924.50,10763.520242,14160.979758
1,46039.49,11052.881586,34986.608414
2,41595.55,10946.790820,30648.759180
3,19403.54,10639.800758,8763.739242
4,21827.90,10492.668341,11335.231659


## 5. Phân tích residual theo phase trong tháng

Binning `month_phase` vào 20 bin trong [0,1], tính residual mean mỗi bin để thấy pattern.

In [ ]:
bins = np.linspace(0, 1, 21)
df["month_phase_bin"] = pd.cut(
    df["month_phase"], bins=bins, labels=False, include_lowest=True,
)

res_mean_by_phase = df.groupby("month_phase_bin")["residual"].mean().reindex(range(len(bins) - 1))
phase_centers = 0.5 * (bins[:-1] + bins[1:])

res_mean_by_phase

,residual
month_phase_bin,
0,-181.310274
1,293.987606
2,268.591700
3,426.087885
4,260.040115
5,110.768311
6,139.012528
7,110.504065
8,-100.528087


## 6. Fit mô hình 2 Gaussian lên pattern residual theo phase

Giả định chu kì lương trung bình có 2 đỉnh trong một tháng, ví dụ khoảng giữa tháng và cuối tháng.

In [ ]:
def two_gaussians(x, a1, mu1, sigma1, a2, mu2, sigma2):
    return (
        a1 * np.exp(-0.5 * ((x - mu1) / sigma1) ** 2)
        + a2 * np.exp(-0.5 * ((x - mu2) / sigma2) ** 2)
    )

x = phase_centers
y_res = np.nan_to_num(res_mean_by_phase.values, nan=0.0)

p0 = [1.0, 0.5, 0.1, 1.0, 0.9, 0.05]

params, cov = curve_fit(two_gaussians, x, y_res, p0=p0, maxfev=20000)
a1, mu1, sigma1, a2, mu2, sigma2 = params

print("Fitted parameters:")
print("a1, mu1, sigma1 =", a1, mu1, sigma1)
print("a2, mu2, sigma2 =", a2, mu2, sigma2)

Fitted parameters:
a1, mu1, sigma1 = -701.9868099934853 1.328727257699509 -0.2679663134718584
a2, mu2, sigma2 = -218.48070133185013 0.6925576571147156 -0.027212343964995375


## 7. Tạo feature `pay_peak_1` và `pay_peak_2` cho toàn bộ tuần

Dùng 2 Gaussian với tham số đã fit để đo intensity của hai đỉnh chu kì lương trung bình.

In [ ]:
df["pay_peak_1"] = np.exp(-0.5 * ((df["month_phase"] - mu1) / sigma1) ** 2)
df["pay_peak_2"] = np.exp(-0.5 * ((df["month_phase"] - mu2) / sigma2) ** 2)

# Chuẩn hóa về [0,1]
df["pay_peak_1"] = df["pay_peak_1"] / df["pay_peak_1"].max()
df["pay_peak_2"] = df["pay_peak_2"] / df["pay_peak_2"].max()

df[["WeekEndDate", "month_phase", "pay_peak_1", "pay_peak_2"]].head()

,WeekEndDate,month_phase,pay_peak_1,pay_peak_2
0,2010-02-05,0.178571,0.000212,3.529400e-78
1,2010-02-12,0.428571,0.007524,3.808525e-21
2,2010-02-19,0.678571,0.111815,9.096622e-01
3,2010-02-26,0.928571,0.695916,4.809179e-17
4,2010-03-05,0.161290,0.000160,1.781871e-83


## 8. Thêm lag và rolling cơ bản

- Lag 1, 4, 52 tuần cho Weekly_Sales
- Rolling mean 4 tuần

In [ ]:
df = df.sort_values(["Store", "Dept", "WeekEndDate"])
group_cols = ["Store", "Dept"]

for lag in [1, 4, 52]:
    df[f"lag_sales_t_{lag}"] = df.groupby(group_cols)["Weekly_Sales"].shift(lag)

df["rolling_mean_sales_4"] = (
    df.groupby(group_cols)["Weekly_Sales"].shift(1).rolling(4).mean()
)

df.head()

,Store,Dept,WeekEndDate,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,week_of_month,y_hat_decomp,residual,month_phase_bin,pay_peak_1,pay_peak_2,lag_sales_t_1,lag_sales_t_4,lag_sales_t_52,rolling_mean_sales_4
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,0.0,...,1,10763.520242,14160.979758,3,0.000212,3.529400e-78,NaN,NaN,NaN,NaN
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,0.0,...,2,11052.881586,34986.608414,8,0.007524,3.808525e-21,24924.50,NaN,NaN,NaN
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,0.0,...,3,10946.790820,30648.759180,13,0.111815,9.096622e-01,46039.49,NaN,NaN,NaN
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,0.0,...,4,10639.800758,8763.739242,18,0.695916,4.809179e-17,41595.55,NaN,NaN,NaN
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,0.0,...,1,10492.668341,11335.231659,3,0.000160,1.781871e-83,19403.54,24924.5,NaN,32990.77


## 9. Train hai model LightGBM để so sánh

- Chia train và validation theo thời gian
- Model A: không dùng pay_peak
- Model B: có pay_peak_1 và pay_peak_2
- So sánh RMSE

In [ ]:
cutoff = "2012-03-01"

train_df = df[df["WeekEndDate"] < cutoff].copy()
valid_df = df[df["WeekEndDate"] >= cutoff].copy()

feature_base = [
    "md_sum",
    "Temperature",
    "Fuel_Price",
    "CPI",
    "Unemployment",
    "lag_sales_t_1",
    "lag_sales_t_4",
    "lag_sales_t_52",
    "rolling_mean_sales_4",
]

# Chỉ giữ những cột thật sự có trong df
feature_base = [c for c in feature_base if c in df.columns]

feature_pay = feature_base + ["pay_peak_1", "pay_peak_2"]

X_train_A = train_df[feature_base].fillna(0)
X_valid_A = valid_df[feature_base].fillna(0)

X_train_B = train_df[feature_pay].fillna(0)
X_valid_B = valid_df[feature_pay].fillna(0)

y_train = train_df["Weekly_Sales"]
y_valid = valid_df["Weekly_Sales"]

print("Feature base:", feature_base)
print("Feature pay:", feature_pay)
print("Train shape A/B:", X_train_A.shape, X_train_B.shape)

Feature base: ['md_sum', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'lag_sales_t_1', 'lag_sales_t_4', 'lag_sales_t_52', 'rolling_mean_sales_4']
Feature pay: ['md_sum', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'lag_sales_t_1', 'lag_sales_t_4', 'lag_sales_t_52', 'rolling_mean_sales_4', 'pay_peak_1', 'pay_peak_2']
Train shape A/B: (317928, 9) (317928, 11)


In [ ]:
model_A = LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.03,
    random_state=42,
)

model_A.fit(X_train_A, y_train)
pred_A = model_A.predict(X_valid_A)
rmse_A = np.sqrt(mean_squared_error(y_valid, pred_A))
print("RMSE Model A (no payday):", rmse_A)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2271
[LightGBM] [Info] Number of data points in the train set: 317928, number of used features: 9
[LightGBM] [Info] Start training from score 16035.433134
RMSE Model A (no payday): 3581.924887977255


In [ ]:
model_B = LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.03,
    random_state=42,
)

model_B.fit(X_train_B, y_train)
pred_B = model_B.predict(X_valid_B)
rmse_B = np.sqrt(mean_squared_error(y_valid, pred_B))
print("RMSE Model B (with payday features):", rmse_B)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011017 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2379
[LightGBM] [Info] Number of data points in the train set: 317928, number of used features: 11
[LightGBM] [Info] Start training from score 16035.433134
RMSE Model B (with payday features): 3324.5808388962346


## 10. Kết luận

- So sánh `RMSE Model A` và `RMSE Model B`
- Nếu RMSE của model B nhỏ hơn đáng kể và ổn định:
  - Chu kì lương 2 đỉnh học từ residual có giá trị trong dự báo
  - Feature `pay_peak_1` và `pay_peak_2` là tín hiệu kinh tế có ý nghĩa, không chỉ là noise

Bạn có thể tiếp tục mở rộng:
- Thêm interaction giữa pay_peak với markdown, holiday
- Thử CatBoost hoặc XGBoost
- Làm cross validation với nhiều cutoff khác nhau.